In [49]:
import requests
import time
def find_best_date(date_data):
    
    dates = []
    date = ""
    a2d = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
           'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    if 'DP' in date_data:
        ymd = [d.strip() for d in date_data['DP'].split(' ')]
        if not ymd or len(ymd[0]) != 4:  # problem with the year
            pass
        else:
            date = ymd.pop(0)
            if ymd:
                mm = a2d.get(ymd.pop(0))
                if mm:
                    date += '-{}'.format(mm)
                    if ymd:
                        dd = ymd.pop(0)
                        if len(dd) <= 2:
                            date += '-{}'.format(dd.zfill(2))
    dates.append(date)
    date = ""
    
    if 'DEP' in date_data:
        date = date_data['DEP']      
        date = date[:4]+"-"+date[4:6]+"-"+date[6:8]
    dates.append(date)
    date = ""
    
    if 'DA' in date_data:
        date = date_data['DA']
        date = date[:4]+"-"+date[4:6]+"-"+date[6:8]
    dates.append(date)
    return dates

def fetch_pubmed(PMID):
    "Takes the number part of PMID and returns title, abstract and authors"

    field2prop = {'TI': 'title', 'AB': 'abstract', 'DP': 'date_published',
                  'DEP': 'date_published', 'DA': 'date_published', 'JT': 'journal',
                  'AU': 'authors', 'CN': 'authors'}
    pub_data = {v: None for v in field2prop.values()}
    pub_data['authors'] = []
    pub_data['date_published'] = {}
    NIHe = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    NIHw = "https://www.ncbi.nlm.nih.gov/pubmed/"
    url = NIHw + PMID
    pub_data['url'] = url
    www = "{NIH}efetch.fcgi?db=pubmed&id={id}&rettype=medline".format(NIH=NIHe, id=PMID)
    # try fetching data 5 times
    time.sleep(1)
    for count in range(50):
        resp = requests.get(www)
        if resp.status_code == 200:
            break
        if resp.status_code == 429:
            time.sleep(5)
            continue

    # parse the text to get the fields
    r = resp.text
    full_text = r.replace('\n      ', ' ')
    for line in full_text.split('\n'):
        if line.strip():
            key, val = [a.strip() for a in line.split('-', 1)]
            if key in field2prop:
                if key in ['DP', 'DEP', 'DA']:
                    pub_data[field2prop[key]][key] = val
                elif key in ['AU', 'CN']:
                    pub_data[field2prop[key]].append(val)
                else:
                    pub_data[field2prop[key]] = val
    # deal with date
    if pub_data['date_published']:  # there is some date data
        pub_data['date_published'] = find_best_date(pub_data['date_published'])
    return {k: v for k, v in pub_data.items() if v is not None}



In [52]:
text = '''20641901
19674507
19668434
19649270
19551993
19526101
19513261
19424844
19383401
19359808
19359804
19258667
19267953
19267952
19253602
19250562
19248620
19241879
19241876
19241870
19227133
19225596
19211999
19204067
19198146
19198125
19181206
19169968
19169610
19162668
19141395
19133616
19132994
19132224
19129909
19126789
19125942
19123674
19122416
19119620
19116667
19116118
19115931
19114361
19112882
19109490
19108711
19108571
19099849
19099848
19094585
19094082
19093005
19092329
19091972
19087328
19086931
19085382
19082934
19080635
19080500
19079043
19077284
19076697
19076439
19076438
19076431
19075734
19074980
19074905
19074031
19074020
19074010
19049602
19066575
19066533
19066093
19066090
19066079
19065317
19061297
19060370
19058517
19056987
19052655
19052286
19050293
19050284
19048539
19046367
19046364
19044202
19044201
19041783
19041781
19041769
19040912
19038214
19037321
19036970
19036966
19033669
19033383
19033098
19032740
19029777
19029058
19024958
19024298
19022766
19021540
19021029
19020019
19018248
19018247
19018000
19017554
19016073
19014432
19014377
19014370
19012257
19011641
19010903
19008946
19008402
19006598
19006454
19005240
19005078
19005077
19004774
19002247
19000382
18995807
18992307
18991953
18991870
18991867
18991353
18991139
18988795
18987829
18987242
18987198
18987185
18987181
18985151
18981663
18981150
18980995
18977454
18975779
18975611
18974604
18973876
18973795
18973588
18973575
18973558
18973551
21158163
18971930
18971641
18971176
18971163
18959796
18958241
18958191
18958155
18957897
18957545
18953891
18952785
18952671
18952589
18952177
18952075
18951957
18950830
18950546
18948169
18948166
18948101
18947400
18947307
18945916
18945906
18945900
18941231
18940855
18940262
18938062
18937355
18931960
18931666
18931067
18930811
18930146
18930091'''


In [53]:

listem = text.split('\n')

for it in listem:
    dicto = fetch_pubmed(it)
    print (";". join([it, dicto['url']] + dicto['date_published']))

20641901;https://www.ncbi.nlm.nih.gov/pubmed/20641901;2004;;2010-07-22
19674507;https://www.ncbi.nlm.nih.gov/pubmed/19674507;2008-05;2009-08-13;
19668434;https://www.ncbi.nlm.nih.gov/pubmed/19668434;2008-12;;
19649270;https://www.ncbi.nlm.nih.gov/pubmed/19649270;2008-08-10;2008-08-10;
19551993;https://www.ncbi.nlm.nih.gov/pubmed/19551993;2008-12-01;;
19526101;https://www.ncbi.nlm.nih.gov/pubmed/19526101;2008-11;;
19513261;https://www.ncbi.nlm.nih.gov/pubmed/19513261;2008;;
19424844;https://www.ncbi.nlm.nih.gov/pubmed/19424844;2008-12;2008-04-23;
19383401;https://www.ncbi.nlm.nih.gov/pubmed/19383401;2008-09;2008-06-10;
19359808;https://www.ncbi.nlm.nih.gov/pubmed/19359808;2008-12;;
19359804;https://www.ncbi.nlm.nih.gov/pubmed/19359804;2008-12;;
19258667;https://www.ncbi.nlm.nih.gov/pubmed/19258667;2008-12;;
19267953;https://www.ncbi.nlm.nih.gov/pubmed/19267953;2008-02;;
19267952;https://www.ncbi.nlm.nih.gov/pubmed/19267952;2008-02;;
19253602;https://www.ncbi.nlm.nih.gov/pubmed/19253602;

19008946;https://www.ncbi.nlm.nih.gov/pubmed/19008946;2008-11;2008-11-14;
19008402;https://www.ncbi.nlm.nih.gov/pubmed/19008402;2008-12;;
19006598;https://www.ncbi.nlm.nih.gov/pubmed/19006598;2008-02;2008-11-13;
19006454;https://www.ncbi.nlm.nih.gov/pubmed/19006454;2008-12;;
19005240;https://www.ncbi.nlm.nih.gov/pubmed/19005240;2008-09;;
19005078;https://www.ncbi.nlm.nih.gov/pubmed/19005078;2008-11-12;;
19005077;https://www.ncbi.nlm.nih.gov/pubmed/19005077;2008-11-12;;
19004774;https://www.ncbi.nlm.nih.gov/pubmed/19004774;2008-11-18;2008-11-12;
19002247;https://www.ncbi.nlm.nih.gov/pubmed/19002247;2008;2008-11-12;
19000382;https://www.ncbi.nlm.nih.gov/pubmed/19000382;2008-12;;
18995807;https://www.ncbi.nlm.nih.gov/pubmed/18995807;2008-11-06;;
18992307;https://www.ncbi.nlm.nih.gov/pubmed/18992307;2008-12-15;2008-10-17;
18991953;https://www.ncbi.nlm.nih.gov/pubmed/18991953;2008-10;;
18991870;https://www.ncbi.nlm.nih.gov/pubmed/18991870;2008-10;;
18991867;https://www.ncbi.nlm.nih.gov/pubm